In [156]:
import requests
import os
import pandas as pd
import numpy as np
import json
from dateutil import relativedelta as rdelta
from datetime import date
from datetime import datetime
from afinn import Afinn

In [22]:
key_df = pd.read_csv("/Users/ajaykliyara/Documents/lc_key.txt",header=None)
key = key_df.iloc[:,:].values[0][0]

In [23]:
s = requests.Session()
s.headers.update({'Authorization': key})
r = s.get('https://api.lendingclub.com/api/investor/v1/loans/listing')


In [30]:
data = json.loads(r.text)

In [46]:
loan_data = data['loans'][0]
loan_data

{'accNowDelinq': 0,
 'accOpenPast24Mths': 1,
 'acceptD': '2016-11-16T16:13:06.000-08:00',
 'addrState': 'NY',
 'addrZip': '112xx',
 'allUtil': 81.8,
 'annualInc': 60000.0,
 'annualIncJoint': None,
 'applicationType': 'INDIVIDUAL',
 'avgCurBal': 2700,
 'bcOpenToBuy': 4398,
 'bcUtil': 84.7,
 'chargeoffWithin12Mths': 0,
 'collections12MthsExMed': 0,
 'creditPullD': '2016-11-14T09:57:22.000-08:00',
 'delinq2Yrs': 0,
 'delinqAmnt': 0.0,
 'desc': None,
 'dti': 12.7,
 'dtiJoint': None,
 'earliestCrLine': '2002-04-13T17:00:00.000-07:00',
 'empLength': 24,
 'empTitle': 'security and command center',
 'expD': '2016-12-31T16:13:07.000-08:00',
 'expDefaultRate': 13.07,
 'ficoRangeHigh': 669,
 'ficoRangeLow': 665,
 'fundedAmount': 9525.0,
 'grade': 'D',
 'homeOwnership': 'RENT',
 'iLUtil': None,
 'id': 93693879,
 'ilsExpD': '2016-12-10T18:00:00.000-08:00',
 'initialListStatus': 'F',
 'inqFi': 0,
 'inqLast12m': 1,
 'inqLast6Mths': 1,
 'installment': 1066.71,
 'intRate': 21.49,
 'investorCount': None

In [63]:
loan_df_ini = pd.DataFrame([loan_data])
loan_df_ini

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totCollAmt,totCurBal,totHiCredLim,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim
0,0,1,2016-11-16T16:13:06.000-08:00,NY,112xx,81.8,60000.0,None,INDIVIDUAL,2700,...,0,24302,29700,15,24302,0.0,28700,0,0,29700


In [58]:
col_list = ['home_ownership', 'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'has_delinquent_hist', 'has_record_hist', 'loan_amnt', 'per_act_open',
       'inq_last_6mths', 'avg_fico', 'revol_util', 'revol_bal', 'dti',
       'years_since_first_credit', 'int_rate', 'emp_length',
       'desc_senti_score', 'zip_median_income', 'zip_pop', 'delinq_2yrs',
       'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'term_ 36 months', 'term_ 60 months', 'purpose_car',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'bankruptcies']

In [68]:
#loan_df = pd.DataFrame(columns=col_list)
loan_df = pd.DataFrame()

In [69]:
loan_df

""


In [70]:
#home ownership
ownership_lookup = {'MORTGAGE': 2, 'NONE': 4, 'OTHER': 3, 'OWN': 1, 'RENT': 0}
loan_df['homeOwnership'] = loan_df_ini['homeOwnership']
loan_df['homeOwnership'].replace(ownership_lookup, inplace=True)
loan_df=loan_df.rename(columns = {'homeOwnership':'home_ownership'})
loan_df['home_ownership']

0    0
Name: home_ownership, dtype: int64

In [80]:
['Verified', 'Not Verified', 'Source Verified']

['Verified', 'Not Verified', 'Source Verified']

In [ ]:
str.upper

In [190]:
#verification status
if(str.upper(loan_df_ini['isIncV'].values[0]) == 'NOT VERIFIED'):
    loan_df['verification_status_Not Verified'] = 1
else:
    loan_df['verification_status_Not Verified'] = 0

if(str.upper(loan_df_ini['isIncV'].values[0]) == 'VERIFIED'):
    loan_df['verification_status_Verified'] = 1
else:
    loan_df['verification_status_Verified'] = 0

if(str.upper(loan_df_ini['isIncV'].values[0]) == 'SOURCE VERIFIED'):
    loan_df['verification_status_Source Verified'] = 1
else:
    loan_df['verification_status_Source Verified'] = 0

loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti,years_since_first_credit,int_rate,emp_length,desc_senti_score,zip_median_income,zip_pop,delinq_2yrs
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7,13,21.49,24,0,46258.984068,65833.432432,0


In [107]:
#has delinq hist
#loan_df_ini['mthsSinceLastDelinq'].apply(lambda s : int(~(s==None) or ~np.isnan(s)))
loan_df['has_delinquent_hist'] = loan_df_ini['mthsSinceLastDelinq'].apply(lambda s : int(s!=None))
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist
0,0,0,0,1,0


In [108]:
#mths_since_last_record
loan_df['has_record_hist'] = loan_df_ini['mthsSinceLastRecord'].apply(lambda s : int(s!=None))
loan_df


,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist
0,0,0,0,1,0,0


In [110]:
loan_df['loan_amnt'] = loan_df_ini['loanAmount']
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt
0,0,0,0,1,0,0,28125.0


In [111]:
#per_act_open
loan_df['per_act_open'] = loan_df_ini.openAcc/loan_df_ini.totalAcc
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open
0,0,0,0,1,0,0,28125.0,0.6


In [112]:
#inqLast6Mths 
loan_df['inq_last_6mths'] = loan_df_ini['inqLast6Mths']
loan_df


,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths
0,0,0,0,1,0,0,28125.0,0.6,1


In [113]:
#avg_fico
loan_df['avg_fico'] = (loan_df_ini['ficoRangeHigh'] +  loan_df_ini['ficoRangeLow'])/2
loan_df


,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico
0,0,0,0,1,0,0,28125.0,0.6,1,667.0


In [114]:
#revol_util
loan_df['revol_util'] = loan_df_ini['revolUtil'] 
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8


In [115]:
#revol_bal revolBal
loan_df['revol_bal'] = loan_df_ini['revolBal'] 
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0


In [116]:
#dti
loan_df['dti'] = loan_df_ini['dti'] 
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7


In [148]:
#years_since_first_credit
loan_df_ini['earliestCrLine'].unique()
ear_cl_df = loan_df_ini['earliestCrLine'].apply(lambda s : datetime.strptime(s[:7],'%Y-%m'))
ear_cl_df.head()

0   2002-04-01
Name: earliestCrLine, dtype: datetime64[ns]

In [149]:
yr_diff = [ rdelta.relativedelta(date(2016,1,1),pd.Timestamp(i).to_pydatetime()).years for i in ear_cl_df.values] 
yr_diff

[13]

In [150]:
ear_cl_df = pd.DataFrame(yr_diff)
ear_cl_df.columns = ['years_since_first_credit']

In [151]:
loan_df['years_since_first_credit'] = ear_cl_df['years_since_first_credit']
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti,years_since_first_credit
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7,13


In [154]:
#int_rate
loan_df['int_rate'] = loan_df_ini['intRate']
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti,years_since_first_credit,int_rate
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7,13,21.49


In [155]:
#emp_length
loan_df['emp_length'] = loan_df_ini['empLength']
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti,years_since_first_credit,int_rate,emp_length
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7,13,21.49,24


In [157]:
#desc_senti_score
afinn = Afinn()

In [166]:
def sent_score(s):
    if s == None:
        return 0
    else:
        afinn.score(s)

s_score = [sent_score(st) for st in loan_df_ini['desc'] ]
s_score

[0]

In [167]:
loan_df['desc_senti_score'] = s_score
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti,years_since_first_credit,int_rate,emp_length,desc_senti_score
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7,13,21.49,24,0


In [168]:
#reading zipcode data
zip_code_agg = pd.read_csv('data/zip3_agg.csv')

In [185]:
#join with final df
loan_df_ini['zip_3'] = [int(zip[:3]) for zip in loan_df_ini.addrZip.values]

In [186]:
join_df = pd.merge(loan_df_ini,zip_code_agg,how='inner',on='zip_3')
join_df

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim,zip_3,Median,Pop
0,0,1,2016-11-16T16:13:06.000-08:00,NY,112xx,81.8,60000.0,None,INDIVIDUAL,2700,...,15,24302,0.0,28700,0,0,29700,112,46258.984068,65833.432432


In [187]:
join_df = join_df.rename(columns={'Median':'zip_median_income','Pop':'zip_pop'})
join_df

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim,zip_3,zip_median_income,zip_pop
0,0,1,2016-11-16T16:13:06.000-08:00,NY,112xx,81.8,60000.0,None,INDIVIDUAL,2700,...,15,24302,0.0,28700,0,0,29700,112,46258.984068,65833.432432


In [188]:
loan_df['zip_median_income'] = join_df['zip_median_income']
loan_df['zip_pop'] = join_df['zip_pop']
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti,years_since_first_credit,int_rate,emp_length,desc_senti_score,zip_median_income,zip_pop
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7,13,21.49,24,0,46258.984068,65833.432432


In [189]:
#delinq2Yrs delinq_2yrs
loan_df['delinq_2yrs'] = loan_df_ini['delinq2Yrs']
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,revol_util,revol_bal,dti,years_since_first_credit,int_rate,emp_length,desc_senti_score,zip_median_income,zip_pop,delinq_2yrs
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,81.8,24302.0,12.7,13,21.49,24,0,46258.984068,65833.432432,0


In [191]:
#grade
#verification status
if(str.upper(loan_df_ini['grade'].values[0]) == 'A'):
    loan_df['grade_A'] = 1
else:
    loan_df['grade_A'] = 0
    
if(str.upper(loan_df_ini['grade'].values[0]) == 'B'):
    loan_df['grade_B'] = 1
else:
    loan_df['grade_B'] = 0
    
    
if(str.upper(loan_df_ini['grade'].values[0]) == 'C'):
    loan_df['grade_C'] = 1
else:
    loan_df['grade_C'] = 0
    

if(str.upper(loan_df_ini['grade'].values[0]) == 'D'):
    loan_df['grade_D'] = 1
else:
    loan_df['grade_D'] = 0
    
if(str.upper(loan_df_ini['grade'].values[0]) == 'E'):
    loan_df['grade_E'] = 1
else:
    loan_df['grade_E'] = 0
    
if(str.upper(loan_df_ini['grade'].values[0]) == 'E'):
    loan_df['grade_E'] = 1
else:
    loan_df['grade_E'] = 0
    
if(str.upper(loan_df_ini['grade'].values[0]) == 'F'):
    loan_df['grade_F'] = 1
else:
    loan_df['grade_F'] = 0
    
if(str.upper(loan_df_ini['grade'].values[0]) == 'G'):
    loan_df['grade_G'] = 1
else:
    loan_df['grade_G'] = 0
    
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,...,zip_median_income,zip_pop,delinq_2yrs,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,...,46258.984068,65833.432432,0,0,0,0,1,0,0,0


In [193]:
#term term_ 36 months term_ 60 months
if(loan_df_ini['term'].values[0] == 36):
    loan_df['term_ 36 months'] = 1
else:
    loan_df['term_ 36 months'] = 0
    
if(loan_df_ini['term'].values[0] == 60):
    loan_df['term_ 60 months'] = 1
else:
    loan_df['term_ 60 months'] = 0

loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,...,delinq_2yrs,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,...,0,0,0,0,1,0,0,0,1,0


In [194]:
#purpose
['credit_card', 'debt_consolidation', 'moving', 'other', 'home_improvement', 'major_purchase', 'car', 'medical', 'small_business', 'educational', 'vacation', 'wedding', 'house', 'renewable_energy']

['credit_card',
 'debt_consolidation',
 'moving',
 'other',
 'home_improvement',
 'major_purchase',
 'car',
 'medical',
 'small_business',
 'educational',
 'vacation',
 'wedding',
 'house',
 'renewable_energy']

In [199]:
if(loan_df_ini['purpose'].values[0] == 'credit_card'):
    loan_df['purpose_credit_card'] = 1
else:
    loan_df['purpose_credit_card'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'debt_consolidation'):
    loan_df['purpose_debt_consolidation'] = 1
else:
    loan_df['purpose_debt_consolidation'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'moving'):
    loan_df['purpose_moving'] = 1
else:
    loan_df['purpose_moving'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'other'):
    loan_df['purpose_other'] = 1
else:
    loan_df['purpose_other'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'home_improvement'):
    loan_df['purpose_home_improvement'] = 1
else:
    loan_df['purpose_home_improvement'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'major_purchase'):
    loan_df['purpose_major_purchase'] = 1
else:
    loan_df['purpose_major_purchase'] = 0

    
if(loan_df_ini['purpose'].values[0] == 'car'):
    loan_df['purpose_car'] = 1
else:
    loan_df['purpose_car'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'medical'):
    loan_df['purpose_medical'] = 1
else:
    loan_df['purpose_medical'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'small_business'):
    loan_df['purpose_small_business'] = 1
else:
    loan_df['purpose_small_business'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'educational'):
    loan_df['purpose_educational'] = 1
else:
    loan_df['purpose_educational'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'vacation'):
    loan_df['purpose_vacation'] = 1
else:
    loan_df['purpose_vacation'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'wedding'):
    loan_df['purpose_wedding'] = 1
else:
    loan_df['purpose_wedding'] = 0
    
    
if(loan_df_ini['purpose'].values[0] == 'house'):
    loan_df['purpose_house'] = 1
else:
    loan_df['purpose_house'] = 0
    
if(loan_df_ini['purpose'].values[0] == 'renewable_energy'):
    loan_df['purpose_renewable_energy'] = 1
else:
    loan_df['purpose_renewable_energy'] = 0
    

loan_df


,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,...,purpose_home_improvement,purpose_major_purchase,purpose_car,purpose_medical,purpose_small_business,purpose_educational,purpose_vacation,purpose_wedding,purpose_house,purpose_renewable_energy
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,...,0,0,0,0,0,0,0,0,0,0


In [200]:
loan_df['purpose_debt_consolidation']

0    1
Name: purpose_debt_consolidation, dtype: int64

In [201]:
#final_df
loan_df

,home_ownership,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,has_delinquent_hist,has_record_hist,loan_amnt,per_act_open,inq_last_6mths,avg_fico,...,purpose_home_improvement,purpose_major_purchase,purpose_car,purpose_medical,purpose_small_business,purpose_educational,purpose_vacation,purpose_wedding,purpose_house,purpose_renewable_energy
0,0,0,0,1,0,0,28125.0,0.6,1,667.0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
#recomputed data
data_recomputed = {}

#### Normalize Data

In [42]:
def test():
    i = input('Enter Loan ID')
    print(i)
    return

In [45]:
test()

Enter Loan ID12
12
